In [ ]:
import pandas as pd
from pyprojroot import here

from lightgbm import LGBMClassifier
from sklearn.metrics import auc, roc_curve, f1_score, log_loss

from tqdm import tqdm
import numpy as np

import shap

import optuna

In [ ]:
data = pd.read_csv(here() / 'data/preprocessed/targets.csv', index_col='Datetime', parse_dates=True)

In [ ]:
TARGET = 'med'
HPO = 0

In [ ]:
y = data[f'Target:Q90_{TARGET}']
y = y.groupby([y.index.date, 
               y.index.hour]).first().unstack()
y = y.iloc[:,8:]
y = (y.sum(axis=1)>=3).astype(int)

In [ ]:
def get_history(series, history, name):
    """
    """
    vectors = []
    
    for h in range(history):
        v = series.shift(h)
        v.name = f'{name}_t-{h}'
        vectors.append(v)

    df = pd.concat(vectors, axis=1)
    df = df[df.columns[::-1]]
    
    return df

In [ ]:
edor = data[f'Target:Edor_{TARGET}']
history_edor = get_history(edor, 8, name='edor')
history_edor = history_edor.iloc[8::24]

In [ ]:
targets = data.columns[data.columns.str.startswith('Target')]

histories = list()

for t in targets:
    v = data[t]
    history = get_history(v, 8, name=v.name)
    histories.append(history)

histories = pd.concat(histories, axis=1)
histories = histories.iloc[8::24]

In [ ]:
#crowding = data[f'Target:Q90_{TARGET}']
#history_crowding = get_history(crowding, 24, name='crowding')
#history_crowding = history_crowding.iloc[8::24]

In [ ]:
dayofweek = pd.Series(history_edor.index.day_of_week, name='dayofweek', index=history_edor.index)
month = pd.Series(history_edor.index.month, name='month', index=history_edor.index)

#fin_holidays = holidays.FIN()
#is_holiday = [x in fin_holidays for x in X.index]
#is_holiday = pd.Series(data = is_holiday, index=X.index)
#X['Holiday'] = is_holiday.astype(int)

In [ ]:
X = pd.concat([history_edor, dayofweek, month], axis=1)

In [ ]:
X.index = X.index.date

In [ ]:
#crowding_t1 = y.shift(1)
#crowding_t1.name = 'crowding_t-1'

crowding_t7 = y.shift(7)
crowding_t7.name = 'crowding_t-7'

crowding_t14 = y.shift(14)
crowding_t14.name = 'crowding_t-14'

X = pd.concat([X, crowding_t7, crowding_t14], axis=1)

In [ ]:
X.columns = X.columns.str.replace('Target:' ,'')

# Expanding window

In [ ]:
class Objective(object):
    
    def __init__(self, y_train, y_val, X_train, X_val):
        self.y_train = y_train
        self.y_val = y_val
        self.X_train = X_train
        self.X_val = X_val

    def __call__(self, trial):
        params = {}

        params['num_leaves'] = trial.suggest_int('num_leaves', 15, 186)
        params['learning_rate'] = trial.suggest_categorical("learning_rate", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
        params['n_estimators'] = trial.suggest_int('n_estimators', 50, 100)
        params['min_child_samples'] = trial.suggest_int('min_child_samples', 10, 120)
        params['subsample'] = trial.suggest_float('subsample', 0.1, 1.00)
        params['history'] = trial.suggest_int('history', 1, self.y_train.shape[0])

        model = LGBMClassifier(class_weight='balanced', 
                               verbose=-1,
                               num_leaves=params['num_leaves'],
                               learning_rate=params['learning_rate'],
                               n_estimators=params['n_estimators'],
                               min_child_samples=params['min_child_samples'],
                               subsample=params['subsample'])

        model.fit(self.X_train.iloc[-params['history']:], 
                  self.y_train.iloc[-params['history']:])
        
        #model.fit(self.X_train, self.y_train)

        y_pred = model.predict(self.X_val)
        error = log_loss(self.y_val, y_pred)
        
        return error

In [ ]:
window = 365
val_size = 30
steps = y.shape[0] - window - 1

y_probs = list()
y_trues = list()
indices = list()

model = LGBMClassifier(class_weight='balanced', verbose=-1)

y_train, y_val = y.iloc[ : window-val_size], y.iloc[window-val_size : window]
X_train, X_val = X.iloc[ : window-val_size], X.iloc[window-val_size : window]

params = {
        'lags' : 28,
        'num_leaves' : 31,
        'learning_rate' : 0.1,
        'n_estimators' : 100,
        'subsample_for_bin' : 200_000,
        'min_child_samples' : 20,
        'subsample' : 1.0
    }

if HPO==1:
    objective = Objective(y_train, y_val, X_train, X_val)
    study = optuna.create_study(direction='minimize')
    study.enqueue_trial(params)
    study.optimize(objective, n_trials=100)
    params = study.best_trial.params

for step in tqdm(range(steps)):
    X_train = X.iloc[ : step+window-val_size]
    y_train = y.iloc[ : step+window-val_size]

    X_val = X.iloc[ step+window-val_size : step+window]
    y_val = y.iloc[ step+window-val_size : step+window]

    if HPO==2:
        objective = Objective(y_train, y_val, X_train, X_val)
        study = optuna.create_study(direction='minimize')
        study.enqueue_trial(params)
        study.optimize(objective, n_trials=100)
        params = study.best_trial.params

    X_test = X.iloc[step+window+1]
    y_test = y.iloc[step+window+1]

    model = LGBMClassifier(class_weight='balanced', 
                           verbose=-1,
                            num_leaves=params['num_leaves'],
                            learning_rate=params['learning_rate'],
                            n_estimators=params['n_estimators'],
                            min_child_samples=params['min_child_samples'],
                            subsample=params['subsample'])

    X_train_val = pd.concat([X_train, X_val])
    y_train_val = pd.concat([y_train, y_val])

    if 'history' in params.keys():
        model.fit(X_train_val.iloc[-params['history']:], 
                y_train_val.iloc[-params['history']:])
    else:
        model.fit(X_train_val, y_train_val)
    
    #model.fit(X_train_val, y_train_val)
    y_prob = model.predict_proba(X_test.values.reshape(1,-1))
    
    y_probs.append(y_prob[0])
    y_trues.append(y_test)

    indices.append(X_test.name)

In [ ]:
y_probs = pd.Series(np.array(y_probs)[:,1], index=indices)
y_trues = np.array(y_trues)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_trues, y_probs)
_auc = auc(fpr, tpr)
_auc

In [ ]:
f1_score(y_trues, y_probs>=0.5)

In [ ]:
y_trues.sum() / y_trues.shape[0]

In [ ]:
y_probs.index.name = 'Datetime'

In [ ]:
y_probs.to_csv(here() / f'data/processed/matrices/prob/h-{TARGET}-lgbm-a-{HPO}.csv')

# Importance

In [ ]:
X_train, X_test = X.iloc[:365], X.iloc[365:]
y_train, y_test = y.iloc[:365], y.iloc[365:]

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

In [ ]:
shap_values

In [ ]:
shap.summary_plot(shap_values, X_test)